Unisco i due file excel contenenti i 3k di dati sulle start-up acquisite o chiuse

In [27]:
import pandas as pd
import numpy as np
import re
from thefuzz import process, fuzz
from src.ingestion import download_from_drive
from tqdm.notebook import tqdm
import os
print(os.getcwd())


/Users/valentinabonato/Desktop/develhope/projects/forecast-italian-firms-success/notebooks


In [28]:
file1 = '../data/file1.csv'
file2 = '../data/file2.csv'

# Leggi i file CSV in DataFrame
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Unisci i due DataFrame
merged_df = pd.concat([df1, df2])

# Visualizza i dati uniti
print(merged_df)

         Organization Name                              Organization Name URL  \
0                  20lines    https://www.crunchbase.com/organization/20lines   
1                  2ASPIRE    https://www.crunchbase.com/organization/2aspire   
2                    2eeet      https://www.crunchbase.com/organization/2eeet   
3                 3 Italia    https://www.crunchbase.com/organization/3-italy   
4     302 Original Content  https://www.crunchbase.com/organization/302-or...   
...                    ...                                                ...   
2338    Axepta BNP Paribas  https://www.crunchbase.com/organization/bnl-po...   
2339                Axioma  https://www.crunchbase.com/organization/axioma...   
2340                Axitea     https://www.crunchbase.com/organization/axitea   
2341                Axylog  https://www.crunchbase.com/organization/axylog...   
2342           AXYZ Design  https://www.crunchbase.com/organization/axyz-d...   

                           

In [29]:
df = merged_df

In [30]:
print(df.iloc[0])
df.head()

Organization Name                                                          20lines
Organization Name URL              https://www.crunchbase.com/organization/20lines
Full Description                 20lines is a website and a mobile app where us...
Website                                                         http://20lines.com
Industries                                        Education, Parenting, Publishing
                                                       ...                        
Total Funding Amount Currency                                                  USD
Total Funding Amount (in USD)                                            1245557.0
Top 5 Investors                  United Ventures, Club Italia Investimenti 2, H...
Number of Lead Investors                                                       NaN
Number of Investors                                                            4.0
Name: 0, Length: 76, dtype: object


,Organization Name,Organization Name URL,Full Description,Website,Industries,Headquarters Location,Description,CB Rank (Company),Founded Date,Founded Date Precision,...,Last Equity Funding Type,Total Equity Funding Amount,Total Equity Funding Amount Currency,Total Equity Funding Amount (in USD),Total Funding Amount,Total Funding Amount Currency,Total Funding Amount (in USD),Top 5 Investors,Number of Lead Investors,Number of Investors
0,20lines,https://www.crunchbase.com/organization/20lines,20lines is a website and a mobile app where us...,http://20lines.com,"Education, Parenting, Publishing","Roncade, Veneto, Italy",20lines is a web and mobile application for us...,"925,584",2012-01-01,year,...,Seed,1245557.0,USD,1245557.0,1245557.0,USD,1245557.0,"United Ventures, Club Italia Investimenti 2, H...",NaN,4.0
1,2ASPIRE,https://www.crunchbase.com/organization/2aspire,2ASPIRE comes from an idea elaborated by the t...,http://www.2aspire.eu,"Apps, Information Technology, Mobile, Mobile A...","Trento, Trentino-Alto Adige, Italy","A security solution for mobile developers, to ...","2,920,717",2016-01-01,year,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2eeet,https://www.crunchbase.com/organization/2eeet,NaN,http://www.2eeet.com,"Apps, Food and Beverage","Treviso, Veneto, Italy",the crispiest app to order pizza,"3,535,266",2016-09-06,day,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3 Italia,https://www.crunchbase.com/organization/3-italy,3 Italia (Tre Italia) was an Italian telecommu...,https://www.tre.it/,Telecommunications,"Trezzano Sul Naviglio, Lombardia, Italy",3 Italia operated as a telecommunications comp...,"1,175,637",1999-11-01,month,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,302 Original Content,https://www.crunchbase.com/organization/302-or...,302 Original Content is an independent film an...,https://302originalcontent.it,"Film, Film Production, Video","Milano, Lombardia, Italy",302 Original Content is an independent film an...,"2,269,428",2021-01-01,year,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df.columns

Index(['Organization Name', 'Organization Name URL', 'Full Description',
       'Website', 'Industries', 'Headquarters Location', 'Description',
       'CB Rank (Company)', 'Founded Date', 'Founded Date Precision',
       'Number of Employees', 'Monthly Visits', 'Headquarters Regions',
       'Postal Code', 'Diversity Spotlight', 'Operating Status', 'Exit Date',
       'Exit Date Precision', 'Closed Date', 'Closed Date Precision',
       'Company Type', 'Twitter', 'Facebook', 'LinkedIn', 'Contact Email',
       'Phone Number', 'Number of Articles', 'Hub Tags', 'Actively Hiring',
       'Number of Sub-Orgs', 'Investor Type', 'Investment Stage',
       'Number of Portfolio Organizations', 'Number of Investments',
       'Number of Lead Investments', 'Number of Diversity Investments',
       'Number of Exits', 'Number of Exits (IPO)', 'Accelerator Program Type',
       'Accelerator Application Deadline', 'Accelerator Duration (in weeks)',
       'Industry Groups', 'Number of Founders', 'F

In [32]:
df["Total Equity Funding Amount (in USD)"]

0       1245557.0
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
2338          NaN
2339          NaN
2340          NaN
2341          NaN
2342          NaN
Name: Total Equity Funding Amount (in USD), Length: 3449, dtype: float64

In [33]:
df = df.drop_duplicates()

In [34]:
df.shape

(2974, 76)

In [35]:
data = df

In [36]:
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 2974 entries, 0 to 2139
Data columns (total 76 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Organization Name                     2974 non-null   object 
 1   Organization Name URL                 2974 non-null   object 
 2   Full Description                      2458 non-null   object 
 3   Website                               2769 non-null   object 
 4   Industries                            2924 non-null   object 
 5   Headquarters Location                 2974 non-null   object 
 6   Description                           2974 non-null   object 
 7   CB Rank (Company)                     2974 non-null   object 
 8   Founded Date                          2605 non-null   object 
 9   Founded Date Precision                2605 non-null   object 
 10  Number of Employees                   2145 non-null   object 
 11  Monthly Visits        

,Organization Name,Organization Name URL,Full Description,Website,Industries,Headquarters Location,Description,CB Rank (Company),Founded Date,Founded Date Precision,...,Last Equity Funding Type,Total Equity Funding Amount,Total Equity Funding Amount Currency,Total Equity Funding Amount (in USD),Total Funding Amount,Total Funding Amount Currency,Total Funding Amount (in USD),Top 5 Investors,Number of Lead Investors,Number of Investors
0,20lines,https://www.crunchbase.com/organization/20lines,20lines is a website and a mobile app where us...,http://20lines.com,"Education, Parenting, Publishing","Roncade, Veneto, Italy",20lines is a web and mobile application for us...,"925,584",2012-01-01,year,...,Seed,1245557.0,USD,1245557.0,1245557.0,USD,1245557.0,"United Ventures, Club Italia Investimenti 2, H...",NaN,4.0
1,2ASPIRE,https://www.crunchbase.com/organization/2aspire,2ASPIRE comes from an idea elaborated by the t...,http://www.2aspire.eu,"Apps, Information Technology, Mobile, Mobile A...","Trento, Trentino-Alto Adige, Italy","A security solution for mobile developers, to ...","2,920,717",2016-01-01,year,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2eeet,https://www.crunchbase.com/organization/2eeet,NaN,http://www.2eeet.com,"Apps, Food and Beverage","Treviso, Veneto, Italy",the crispiest app to order pizza,"3,535,266",2016-09-06,day,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3 Italia,https://www.crunchbase.com/organization/3-italy,3 Italia (Tre Italia) was an Italian telecommu...,https://www.tre.it/,Telecommunications,"Trezzano Sul Naviglio, Lombardia, Italy",3 Italia operated as a telecommunications comp...,"1,175,637",1999-11-01,month,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,302 Original Content,https://www.crunchbase.com/organization/302-or...,302 Original Content is an independent film an...,https://302originalcontent.it,"Film, Film Production, Video","Milano, Lombardia, Italy",302 Original Content is an independent film an...,"2,269,428",2021-01-01,year,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
